In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder.appName("").getOrCreate()

In [6]:
offenses_path = 'C:/Users/Досжан/Desktop/ca_offenses_by_campus.csv'
law_enforcement_path = 'C:/Users/Досжан/Desktop/ca_law_enforcement_by_campus.csv'


In [7]:
offenses_path = 'ca_offenses_by_campus.csv'
# Load the dataset into a Spark DataFrame
offenses_df = spark.read.csv(offenses_path, header=True, inferSchema=True)

In [8]:
law_enforcement_path = 'ca_law_enforcement_by_campus.csv'
# Load the dataset into a Spark DataFrame
law_enforcement_df = spark.read.csv(law_enforcement_path, header=True, inferSchema=True)

In [9]:
print("Schema for offenses_df:")
offenses_df.printSchema()

print("\nSchema for law_enforcement_df:")
law_enforcement_df.printSchema()

Schema for offenses_df:
root
 |-- University/College: string (nullable = true)
 |-- Campus: string (nullable = true)
 |-- Student enrollment: string (nullable = true)
 |-- Violent crime: integer (nullable = true)
 |-- Murder and nonnegligent manslaughter: integer (nullable = true)
 |-- Rape (revised definition): integer (nullable = true)
 |-- Rape (legacy definition): string (nullable = true)
 |-- Robbery: integer (nullable = true)
 |-- Aggravated assault: integer (nullable = true)
 |-- Property crime: integer (nullable = true)
 |-- Burglary: integer (nullable = true)
 |-- Larceny-theft: integer (nullable = true)
 |-- Motor vehicle theft: integer (nullable = true)
 |-- Arson: integer (nullable = true)


Schema for law_enforcement_df:
root
 |-- University/College : string (nullable = true)
 |-- Campus: string (nullable = true)
 |-- Student: string (nullable = true)



In [10]:
joined_df = offenses_df.join(law_enforcement_df, offenses_df['campus'] == law_enforcement_df['campus'], 'inner')

In [11]:
print("Columns in offenses_df:", offenses_df.columns)
print("Columns in law_enforcement_df:", law_enforcement_df.columns)

Columns in offenses_df: ['University/College', 'Campus', 'Student enrollment', 'Violent crime', 'Murder and nonnegligent manslaughter', 'Rape (revised definition)', 'Rape (legacy definition)', 'Robbery', 'Aggravated assault', 'Property crime', 'Burglary', 'Larceny-theft', 'Motor vehicle theft', 'Arson']
Columns in law_enforcement_df: ['University/College ', 'Campus', 'Student']


In [17]:
# Use Spark SQL to find the top 10 campuses
joined_df.createOrReplaceTempView("campus_data")
top_10_campuses = spark.sql("""
    SELECT cd.Campus AS campus,
           (cd.`Violent crime` / cd.`Student enrollment`) * 100 AS violent_crime_rate_per_100_students
    FROM campus_data cd
    ORDER BY violent_crime_rate_per_100_students DESC
    LIMIT 10
""")


In [ ]:
top_10_campuses.show()

+---------------+-----------------------------------+
|         campus|violent_crime_rate_per_100_students|
+---------------+-----------------------------------+
|         Pomona|                               NULL|
|    Bakersfield|                               NULL|
|Channel Islands|                               NULL|
|Dominguez Hills|                               NULL|
|       East Bay|                               NULL|
|         Fresno|                               NULL|
|    Los Angeles|                               NULL|
|   Monterey Bay|                               NULL|
|     Northridge|                               NULL|
|     Sacramento|                               NULL|
+---------------+-----------------------------------+



In [18]:
# Write the top 10 results to a CSV file
top_10_campuses.write.csv("path/to/top_10_campuses.csv", header=True)


In [19]:
print("Top 10 Campuses by Highest Violent Crime Rate:")
top_10_campuses.show()

# Display the joined DataFrame with calculated crime rates
print("Joined DataFrame with Calculated Crime Rates:")
joined_df.show()
